In [1]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler

In [2]:
def data_concat(df,path):
    for p in path:
        data = pd.read_csv(p,header=None)
        df = pd.concat([df,data],axis=0)
    return df

In [3]:
# 取PSD特征
def psd_feature(df):
    m=df.shape[1]
    n=100   #每100个频点作为一个子带
    signal_feature = pd.DataFrame()
    for i in range(m/n):
        df_n=df.iloc[:,range(n*i,n*(i+1))]
        df_n = 10**(df_n/10)
        mean=df_n.mean(axis=1)
        var = df_n.var(axis=1)
        mean=mean.map(lambda x :math.log(x,10)*10)
        signal_feature['psd_mean_'+str(i)] = mean
        signal_feature['psd_var_'+str(i)] = var
    return signal_feature

In [4]:
# 取PSD差分特征
def psd_cha_feature(df):
    m=df.shape[1]
    n=100  #每100个频点作为一个子带
    signal_feature = pd.DataFrame()
    for i in range(m/n):
        df_n=df.iloc[:,range(n*i,n*(i+1))]
        mean=df_n.mean(axis=1)    # mean 代表该频段的平均上升下降次数
        #mean=mean.map(lambda x:0 if x>-0.5 else 1) # 二次离散化
        signal_feature['psd_cha_'+str(i)] = mean
    return signal_feature

In [5]:
def convert_cha(sa):
    if (sa>0):
        return 1
    else:
        return -1

In [6]:
# 取差值（斜率）作为特征
def cha(df):
    n=df.shape[1]
    ret = pd.DataFrame()
    for i in range(n-1):
        d=df.iloc[:,i+1]-df.iloc[:,i]
        ret[i]=d
    ret[n-1]=0
    return ret

In [9]:
path_all=['./matlab/data_418/data_UAV_f_-95-60.csv','./matlab/data_418/data_UAV_f_-95-70.csv','./matlab/data_418/data_UAV_f_-95-80.csv',\
         './matlab/data_418/data_UAV_f_-95-90.csv','./matlab/data_418/data_UAV_f_-95-100.csv','./matlab/data_418/data_noise_f_-95.csv']

In [33]:
all_data = pd.DataFrame()
all_data = data_concat(all_data,path_all)
all_data.shape

(5000, 5000)

In [34]:
all_data=all_data.iloc[:,range(1200,4200)]
all_data.shape

(5000, 3000)

In [35]:
all_data_cha = cha(all_data)
all_data_cha.shape

(5000, 3000)

In [36]:
all_data_cha = smooth(all_data_cha)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: object too deep for desired array

In [37]:
all_data.index=range(all_data.shape[0])
all_data_cha.index=range(all_data_cha.shape[0])

In [38]:
# train_feature 为最终的训练特征
all_feature = pd.DataFrame()

In [39]:
psd = psd_feature(all_data)
psd = psd.sort_index(axis=1)
#psd_mean_feature=psd.iloc[:,range(30)].applymap(convert_data)
#psd_var_feature=psd.iloc[:,range(30,60)].applymap(convert_var)

In [40]:
all_data_cha_cate = all_data_cha.applymap(convert_cha)
psd_cha = psd_cha_feature(all_data_cha_cate)
psd_cha.shape

(5000, 30)

In [41]:
all_feature = pd.concat([psd,psd_cha],axis=1)
all_feature.shape

(5000, 90)

In [42]:
all_feature

,psd_mean_0,psd_mean_1,psd_mean_10,psd_mean_11,psd_mean_12,psd_mean_13,psd_mean_14,psd_mean_15,psd_mean_16,psd_mean_17,...,psd_cha_20,psd_cha_21,psd_cha_22,psd_cha_23,psd_cha_24,psd_cha_25,psd_cha_26,psd_cha_27,psd_cha_28,psd_cha_29
0,-86.286091,-81.951133,-56.985271,-57.000315,-56.990325,-56.980567,-56.981898,-56.991559,-57.000213,-56.988101,...,-0.06,-0.02,0.04,-0.12,0.02,0.00,-0.02,-0.30,-0.68,0.00
1,-86.831519,-81.846822,-56.985261,-56.986964,-56.984833,-56.990115,-56.988621,-56.998701,-57.001987,-56.994449,...,-0.06,-0.02,0.00,0.00,0.00,0.04,0.02,-0.28,-0.70,-0.02
2,-86.686625,-81.776674,-56.986535,-56.995779,-56.991479,-56.989484,-56.991342,-56.992551,-57.004421,-56.993088,...,-0.02,-0.02,-0.04,0.02,0.08,-0.04,-0.02,-0.20,-0.72,-0.14
3,-86.347749,-81.897480,-56.990806,-56.994638,-56.987805,-56.985261,-56.990830,-56.995200,-56.993915,-56.993778,...,0.06,0.02,-0.04,0.02,0.02,0.00,-0.06,-0.26,-0.56,-0.20
4,-86.340790,-82.075834,-56.981740,-56.988928,-57.010215,-56.987617,-56.986052,-56.992256,-57.001535,-56.988503,...,0.04,0.02,0.02,-0.02,-0.04,-0.02,0.04,-0.30,-0.66,-0.14
5,-86.550436,-82.008550,-56.998341,-57.008973,-56.989296,-56.973410,-57.004881,-56.987949,-56.991768,-56.994901,...,-0.02,-0.08,-0.06,0.06,-0.02,-0.02,-0.02,-0.26,-0.72,-0.10
6,-86.099571,-81.883212,-56.999944,-56.988882,-56.981321,-56.999852,-56.983803,-56.996150,-56.985747,-57.000701,...,-0.04,0.00,-0.02,-0.02,-0.04,-0.04,-0.04,-0.22,-0.64,-0.12
7,-86.235206,-81.947525,-56.988968,-56.990119,-56.995740,-56.983113,-56.972050,-56.983242,-56.988653,-56.991452,...,0.06,-0.12,-0.08,-0.10,-0.02,0.04,-0.06,-0.26,-0.72,-0.16
8,-86.266560,-81.970567,-56.987857,-57.001170,-56.986859,-57.013210,-57.001834,-56.980886,-56.984780,-56.985921,...,0.08,0.06,-0.02,-0.10,-0.08,-0.04,0.00,-0.20,-0.70,-0.08
9,-86.168936,-81.988484,-56.994517,-57.001463,-57.001992,-56.989909,-56.982717,-56.972145,-56.986137,-56.985786,...,-0.06,0.00,-0.14,0.06,0.02,0.02,-0.08,-0.24,-0.66,-0.12


In [32]:
all_feature.iloc[range(2000,4000),:]

,psd_mean_0,psd_mean_1,psd_mean_10,psd_mean_11,psd_mean_12,psd_mean_13,psd_mean_14,psd_mean_15,psd_mean_16,psd_mean_17,...,psd_cha_20,psd_cha_21,psd_cha_22,psd_cha_23,psd_cha_24,psd_cha_25,psd_cha_26,psd_cha_27,psd_cha_28,psd_cha_29
2000,-93.418156,-94.636598,-93.739562,-95.135943,-95.084010,-93.980524,-93.846772,-95.884301,-93.750245,-94.486244,...,0.04,0.00,0.04,0.12,-0.04,-0.06,0.08,-0.08,-0.02,0.12
2001,-95.724353,-94.639217,-93.520001,-94.686000,-94.709247,-94.757865,-94.123674,-94.334829,-94.525916,-95.686285,...,-0.02,0.04,-0.02,0.06,-0.02,0.00,0.02,0.02,-0.04,-0.06
2002,-94.267179,-94.660636,-94.563329,-94.760118,-94.112036,-95.337229,-93.705156,-94.926459,-94.720441,-94.815611,...,0.04,-0.08,0.06,0.04,-0.02,-0.16,-0.02,0.04,-0.02,0.00
2003,-95.506984,-94.527759,-93.892949,-95.587148,-94.102167,-93.522360,-95.255182,-94.266158,-95.130220,-94.548436,...,0.06,-0.02,0.06,-0.08,0.04,0.06,-0.04,0.06,0.02,-0.10
2004,-94.666213,-95.068885,-94.802693,-94.189662,-94.187060,-94.422241,-93.240634,-94.804498,-95.051701,-94.145100,...,-0.06,-0.04,-0.02,0.10,-0.06,0.00,0.00,-0.10,0.02,-0.04
2005,-94.700424,-94.161756,-95.301633,-93.885038,-94.844906,-94.593345,-95.244157,-94.472582,-94.218010,-93.714769,...,-0.10,0.08,-0.02,-0.02,-0.04,0.10,0.02,0.12,0.08,0.06
2006,-95.152920,-94.393960,-94.172689,-93.895141,-94.101147,-93.412744,-94.430327,-94.771102,-95.062518,-94.923346,...,0.00,-0.04,-0.02,0.00,-0.02,0.00,0.02,0.04,0.00,0.00
2007,-94.674585,-94.441384,-94.303616,-94.400313,-94.472358,-94.422634,-94.434097,-95.513130,-94.403685,-93.986131,...,-0.06,-0.02,0.08,0.06,-0.06,0.12,-0.06,-0.08,0.02,0.08
2008,-94.264233,-93.483894,-94.824949,-94.157608,-94.356896,-94.726865,-94.528583,-94.102399,-94.581059,-94.470208,...,0.00,0.08,0.08,-0.02,-0.06,0.08,0.06,0.06,0.02,-0.14
2009,-94.635406,-95.358420,-94.253329,-94.382380,-94.640698,-94.360574,-93.486624,-94.329635,-94.013827,-94.617746,...,0.08,-0.04,-0.04,-0.06,-0.02,0.12,0.02,0.06,-0.08,-0.06


In [43]:
data_all=MinMaxScaler().fit_transform(all_feature) #normalization 得到numpy array

In [44]:
all_data = pd.DataFrame(data_all)

In [45]:
all_data.columns=all_feature.columns

In [46]:
all_data['label']=[1]*2500+[0]*2500
all_data

,psd_mean_0,psd_mean_1,psd_mean_10,psd_mean_11,psd_mean_12,psd_mean_13,psd_mean_14,psd_mean_15,psd_mean_16,psd_mean_17,...,psd_cha_21,psd_cha_22,psd_cha_23,psd_cha_24,psd_cha_25,psd_cha_26,psd_cha_27,psd_cha_28,psd_cha_29,label
0,0.965391,0.979161,0.999653,0.999293,0.999486,0.999828,0.999737,0.999521,0.999408,0.999487,...,0.500000,0.636364,0.238095,0.545455,0.50,0.476190,0.137931,0.153846,0.653846,1
1,0.922620,0.985188,0.999653,0.999611,0.999619,0.999598,0.999577,0.999349,0.999365,0.999336,...,0.500000,0.545455,0.523810,0.500000,0.60,0.571429,0.172414,0.134615,0.615385,1
2,0.933982,0.989241,0.999622,0.999401,0.999458,0.999613,0.999513,0.999497,0.999307,0.999368,...,0.500000,0.454545,0.571429,0.681818,0.40,0.476190,0.310345,0.115385,0.384615,1
3,0.960556,0.982261,0.999519,0.999428,0.999547,0.999715,0.999525,0.999433,0.999559,0.999352,...,0.590909,0.454545,0.571429,0.545455,0.50,0.380952,0.206897,0.269231,0.269231,1
4,0.961102,0.971956,0.999738,0.999564,0.999006,0.999658,0.999638,0.999504,0.999376,0.999477,...,0.590909,0.590909,0.476190,0.409091,0.45,0.619048,0.137931,0.173077,0.384615,1
5,0.944662,0.975843,0.999338,0.999087,0.999511,1.000000,0.999191,0.999607,0.999611,0.999325,...,0.363636,0.409091,0.666667,0.454545,0.45,0.476190,0.206897,0.115385,0.461538,1
6,0.980018,0.983085,0.999299,0.999565,0.999704,0.999364,0.999692,0.999410,0.999755,0.999187,...,0.545455,0.500000,0.476190,0.409091,0.40,0.428571,0.275862,0.192308,0.423077,1
7,0.969382,0.979369,0.999564,0.999536,0.999356,0.999767,0.999971,0.999720,0.999686,0.999407,...,0.272727,0.363636,0.285714,0.454545,0.60,0.380952,0.206897,0.115385,0.346154,1
8,0.966923,0.978038,0.999590,0.999273,0.999570,0.999043,0.999263,0.999777,0.999778,0.999538,...,0.681818,0.500000,0.285714,0.318182,0.40,0.523810,0.310345,0.134615,0.500000,1
9,0.974578,0.977003,0.999430,0.999266,0.999205,0.999603,0.999717,0.999987,0.999746,0.999541,...,0.545455,0.227273,0.666667,0.545455,0.55,0.333333,0.241379,0.173077,0.423077,1


In [47]:
all_data.to_csv('train_data_node1_0418.csv',index=False)

In [18]:
def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    r"""Smooth (and optionally differentiate) data with a Savitzky-Golay filter.
    The Savitzky-Golay filter removes high frequency noise from data.
    It has the advantage of preserving the original shape and
    features of the signal better than other types of filtering
    approaches, such as moving averages techniques.
    Parameters
    ----------
    y : array_like, shape (N,)
        the values of the time history of the signal.
    window_size : int
        the length of the window. Must be an odd integer number.
    order : int
        the order of the polynomial used in the filtering.
        Must be less then `window_size` - 1.
    deriv: int
        the order of the derivative to compute (default = 0 means only smoothing)
    Returns
    -------
    ys : ndarray, shape (N)
        the smoothed signal (or it's n-th derivative).
    Notes
    -----
    The Savitzky-Golay is a type of low-pass filter, particularly
    suited for smoothing noisy data. The main idea behind this
    approach is to make for each point a least-square fit with a
    polynomial of high order over a odd-sized window centered at
    the point.
    Examples
    --------
    t = np.linspace(-4, 4, 500)
    y = np.exp( -t**2 ) + np.random.normal(0, 0.05, t.shape)
    ysg = savitzky_golay(y, window_size=31, order=4)
    import matplotlib.pyplot as plt
    plt.plot(t, y, label='Noisy signal')
    plt.plot(t, np.exp(-t**2), 'k', lw=1.5, label='Original signal')
    plt.plot(t, ysg, 'r', label='Filtered signal')
    plt.legend()
    plt.show()
    References
    ----------
    .. [1] A. Savitzky, M. J. E. Golay, Smoothing and Differentiation of
       Data by Simplified Least Squares Procedures. Analytical
       Chemistry, 1964, 36 (8), pp 1627-1639.
    .. [2] Numerical Recipes 3rd Edition: The Art of Scientific Computing
       W.H. Press, S.A. Teukolsky, W.T. Vetterling, B.P. Flannery
       Cambridge University Press ISBN-13: 9780521880688
    """
    import numpy as np
    from math import factorial
    
    try:
        window_size = np.abs(np.int(window_size))
        order = np.abs(np.int(order))
    except ValueError, msg:
        raise ValueError("window_size and order have to be of type int")
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')

In [19]:
def smooth(df):
    for i in range(len(df)):
        y = df.ix[i].values
        yhat = savitzky_golay(y, 101, 3)
        df.ix[i]=yhat
    return df